# Image Proprocessing

In [27]:
import os
import shutil

In [28]:
def read_all_file(path):
    output = os.listdir(path)
    file_list = []

    for i in output:
        if os.path.isdir(path+"/"+i): 
            file_list.extend(read_all_file(path+"/"+i)) 
        elif os.path.isfile(path+"/"+i):
            file_list.append(path+"/"+i)

    return file_list

In [29]:
def copy_all_file(file_list, new_path):
    for src_path in tqdm(file_list):
        file = src_path.split("/")[-1]
        shutil.copyfile(src_path, new_path+"/"+file)

In [32]:
src_path = r"C:\HUFS_capstone\dataset\임서연\dog_nose_FINISH" 
new_path = r"C:\HUFS_capstone\dataset\임서연\Dataset_SH\train_images(origin)"

file_list = read_all_file(src_path)
copy_all_file(file_list, new_path)

## 파일 확장자 jpg 로 통일

In [33]:
import os
import glob

files = glob.glob("C:/HUFS_capstone/dataset/임서연/Dataset_SH/train_images(origin)/*.jpeg")

for name in files:
    if not os.path.isdir(name):
        src = os.path.splitext(name)
        os.rename(name, src[0] + '.jpg')
    print(name)

C:/HUFS_capstone/dataset/임서연/Dataset_SH/train_images(origin)\n100003.jpeg
C:/HUFS_capstone/dataset/임서연/Dataset_SH/train_images(origin)\n100026.jpeg
C:/HUFS_capstone/dataset/임서연/Dataset_SH/train_images(origin)\n100029.jpeg
C:/HUFS_capstone/dataset/임서연/Dataset_SH/train_images(origin)\n100031.jpeg
C:/HUFS_capstone/dataset/임서연/Dataset_SH/train_images(origin)\n100034.jpeg
C:/HUFS_capstone/dataset/임서연/Dataset_SH/train_images(origin)\n100036.jpeg
C:/HUFS_capstone/dataset/임서연/Dataset_SH/train_images(origin)\n100084.jpeg
C:/HUFS_capstone/dataset/임서연/Dataset_SH/train_images(origin)\n100085.jpeg
C:/HUFS_capstone/dataset/임서연/Dataset_SH/train_images(origin)\n100114.jpeg
C:/HUFS_capstone/dataset/임서연/Dataset_SH/train_images(origin)\n100115.jpeg
C:/HUFS_capstone/dataset/임서연/Dataset_SH/train_images(origin)\n100135.jpeg
C:/HUFS_capstone/dataset/임서연/Dataset_SH/train_images(origin)\n100140.jpeg
C:/HUFS_capstone/dataset/임서연/Dataset_SH/train_images(origin)\n100155.jpeg
C:/HUFS_capstone/dataset/임서연/Dataset_S

C:/HUFS_capstone/dataset/임서연/Dataset_SH/train_images(origin)\n165460.jpeg
C:/HUFS_capstone/dataset/임서연/Dataset_SH/train_images(origin)\n165488.jpeg
C:/HUFS_capstone/dataset/임서연/Dataset_SH/train_images(origin)\n165504.jpeg
C:/HUFS_capstone/dataset/임서연/Dataset_SH/train_images(origin)\n165510.jpeg
C:/HUFS_capstone/dataset/임서연/Dataset_SH/train_images(origin)\n165531.jpeg
C:/HUFS_capstone/dataset/임서연/Dataset_SH/train_images(origin)\n165549.jpeg
C:/HUFS_capstone/dataset/임서연/Dataset_SH/train_images(origin)\n165657.jpeg
C:/HUFS_capstone/dataset/임서연/Dataset_SH/train_images(origin)\n165698.jpeg
C:/HUFS_capstone/dataset/임서연/Dataset_SH/train_images(origin)\n165772.jpeg
C:/HUFS_capstone/dataset/임서연/Dataset_SH/train_images(origin)\n165790.jpeg
C:/HUFS_capstone/dataset/임서연/Dataset_SH/train_images(origin)\n165818.jpeg
C:/HUFS_capstone/dataset/임서연/Dataset_SH/train_images(origin)\n165833.jpeg
C:/HUFS_capstone/dataset/임서연/Dataset_SH/train_images(origin)\n165873.jpeg
C:/HUFS_capstone/dataset/임서연/Dataset_S

# Data Augmentation

In [1]:
import os
import cv2
import time
import numpy as np
from tqdm import tqdm
import imgaug as ia
import imgaug.augmenters as iaa

## Define required custom functions

In [2]:
def Yolo2Coord(size, box):
    category = int(box[0]) # class
    x_center, y_center, width, height = float(box[1]), float(box[2]), float(box[3]), float(box[4])
    img_width, img_height = size[0], size[1]
    x_min = int((x_center - width/2.) * img_width)
    x_max = int((x_center + width/2.) * img_width)
    y_min = int((y_center - height/2.) * img_height)
    y_max = int((y_center + height/2.) * img_height)
    return [category, x_min, y_min, x_max, y_max]

In [3]:
def Pascal2Yolo(box, size):
    image_width, image_height = size[0], size[1]
    # xmin, ymin, xmax, ymax
    x_center = ((box[2] + box[0]) / 2) / (image_width)
    y_center = ((box[3] + box[1]) / 2) / (image_height)
    width = (box[2] - box[0]) / (image_width)
    height = (box[3] - box[1]) / (image_height)
    return [str(x_center), str(y_center), str(width), str(height)]

In [4]:
def after_box(bbs):
    before = []
    for i in range(len(bbs)):
        x_min = bbs[i][0][0]
        y_min = bbs[i][0][1]
        x_max = bbs[i][1][0]
        y_max = bbs[i][1][1]
    before.append([x_min, y_min, x_max, y_max])
    return before

# DI LEE

## Route Setting

In [5]:
# 경로 설정
root_path = r"C:/HUFS_capstone/dataset/dataset_DI/" #원천 이미지 & 라벨링 디렉토리
image_path = root_path + "train_images(origin)/" # 원천 이미지 디렉토리
label_path = root_path + "train_labels(origin)/" # 원천 이미지 라벨 디렉토리

images = list(os.listdir(image_path))
print(len(images), images[:5], sep="\n")
labels = list(os.listdir(label_path))
print(len(labels), labels[:5], sep="\n")

1025
['n109535.jpg', 'n109537.jpg', 'n109538.jpg', 'n109543.jpg', 'n109546.jpg']
1025
['n109535.txt', 'n109537.txt', 'n109538.txt', 'n109543.txt', 'n109546.txt']


## Augmentation

## first

#### Categorize Augmented Data

In [6]:
aug_name = "aug__" # 원하는 파일명 적기

In [7]:
for img in tqdm(images):
#     print(img)
    image = cv2.imread(image_path + img) # 이미지 불러오기
#     print(image)
    label = img[:-4] + ".txt" # 라벨링 데이터 불러오기
#     print(label)
    height, width = image.shape[:2] # 원천 이미지 픽셀 불러오기 (height, weight, dimension)
    image_size = [width, height] # cv2는 height * weight를 출력하기 때문에 width, height형식으로 변환

    # bounding box 라벨 저장
    label_list = [] # 텍스트 파일에 있는 라벨을 담을 빈 리스트

    with open(label_path + label, "r") as f:
        labs = f.readlines()
        for lab in labs:
              label_list.append(lab.split()) # 라벨들을 리스트에 저장
        f.close()

    new_bounding_boxes = []

    for box in label_list:
        new_bounding_boxes.append(Yolo2Coord(image_size, box))
    cate = str(new_bounding_boxes[0][0])

    # augmentation 할 때 쓰는 BoundingBox 타입으로 변환
    ia_bounding_boxes = []

    for box in new_bounding_boxes:
        ia_bounding_boxes.append(ia.BoundingBox(x1 = box[1], y1 = box[2], x2 = box[3], y2 = box[4]))

    # BoundingBoxesOnImage([BoundingBox(x1= ~, y1= ~, x2= ~, y2= ~, label=None)형식

    # augmentation
    bbs = ia.BoundingBoxesOnImage(ia_bounding_boxes, shape = image.shape)

    seq = iaa.Sequential([
        iaa.AddToHueAndSaturation((-20, 20), per_channel = True),
        iaa.Sometimes(0.5, iaa.Invert(0.1, per_channel = True)),
        iaa.Sometimes(0.2, iaa.Grayscale(alpha = (0.0, 0.5))),
        iaa.OneOf([iaa.Flipud(0.3), # 30%의 이미지 상하 반전
                 iaa.Fliplr(0.4)]), # 40%의 이미지 좌우 반전
#         iaa.Sometimes(0.5, iaa.Superpixels(p_replace = (0, 0.5), n_segments = (20, 100))), # Superpixels: 비슷한 색/밝기를 가진 pixel 그룹화
        iaa.Sometimes(0.5, iaa.LinearContrast((0.75, 1.5), per_channel = 0.5)), # LinearContrast: 이미지 일정 부분의 톤 차이를 돋보이게 만듦, 강한 대비
        iaa.Sometimes(0.5, iaa.Affine(shear = (-4, 4), # Affine: 이미지 회전
                                    rotate = (-40, 40), fit_output = True)), # fit_output: True 값을 적용해야 회전시켜도 이미지가 잘리지 않음
#         iaa.Sometimes(0.5, iaa.Emboss(alpha = (0, 0.5), strength = (0, 1.0))), # Emboss: 이미지 입체감 부여
#         iaa.OneOf([iaa.Grayscale(alpha = (0.0, 0.3)),
#                 iaa.MedianBlur(k = (3, 11))]), # MedianBlur: 이미지 블러 처리
        iaa.Sometimes(0.5, iaa.Sharpen(alpha = (0, 0.5), lightness = (0.75, 1.5)))], # Sharpen: 흐릿한 이미지를 선명하게 만듦
        random_order = True)

    image_aug, bbs_aug = seq(image = image, bounding_boxes = bbs) # image_aug : 증강된 이미지, bbs_aug : 증강이 적용된 bounding box 좌표

    cv2.imwrite(root_path + "aug_img/"  + aug_name + img, image_aug) # 증강 이미지 저장
    # 증강된 사진 좌표
    resized = image_aug.shape
    rotated_size = [resized[1], resized[0]] # height, width -> width, height

    augmented_coords = after_box(bbs_aug)

    yolos = []

    for i in range(len(augmented_coords)):
        a = Pascal2Yolo(augmented_coords[i], rotated_size)
        final = cate + " " + " ".join(a)
        yolos.append(final)
  
    # print(yolos)
    # print("===================================================================================")


    f = open(root_path + "aug_label/" + aug_name + label, "w", encoding = "utf-8", newline='') # 증강 라벨 저장
  
    for yolo in yolos:
        f.write(yolo)
        f.write("\n")
    f.close()
  

100%|██████████████████████████████████████████████████████████████████████████████| 1025/1025 [00:26<00:00, 38.63it/s]


## second

#### Categorize Augmented Data

In [8]:
aug_name = "aug_1_" # 원하는 파일명 적기

In [9]:
for img in tqdm(images):
#     print(img)
    image = cv2.imread(image_path + img) # 이미지 불러오기
#     print(image)
    label = img[:-4] + ".txt" # 라벨링 데이터 불러오기
#     print(label)
    height, width = image.shape[:2] # 원천 이미지 픽셀 불러오기 (height, weight, dimension)
    image_size = [width, height] # cv2는 height * weight를 출력하기 때문에 width, height형식으로 변환

    # bounding box 라벨 저장
    label_list = [] # 텍스트 파일에 있는 라벨을 담을 빈 리스트

    with open(label_path + label, "r") as f:
        labs = f.readlines()
        for lab in labs:
              label_list.append(lab.split()) # 라벨들을 리스트에 저장
        f.close()

    new_bounding_boxes = []

    for box in label_list:
        new_bounding_boxes.append(Yolo2Coord(image_size, box))
    cate = str(new_bounding_boxes[0][0])

    # augmentation 할 때 쓰는 BoundingBox 타입으로 변환
    ia_bounding_boxes = []

    for box in new_bounding_boxes:
        ia_bounding_boxes.append(ia.BoundingBox(x1 = box[1], y1 = box[2], x2 = box[3], y2 = box[4]))

    # BoundingBoxesOnImage([BoundingBox(x1= ~, y1= ~, x2= ~, y2= ~, label=None)형식

    # augmentation
    bbs = ia.BoundingBoxesOnImage(ia_bounding_boxes, shape = image.shape)

    seq = iaa.Sequential([
        iaa.AddToHueAndSaturation((-20, 20), per_channel = True),
        iaa.Sometimes(0.5, iaa.Invert(0.1, per_channel = True)),
        iaa.Sometimes(0.2, iaa.Grayscale(alpha = (0.0, 0.5))),
        iaa.OneOf([iaa.Flipud(0.3), # 30%의 이미지 상하 반전
                 iaa.Fliplr(0.4)]), # 40%의 이미지 좌우 반전
#         iaa.Sometimes(0.5, iaa.Superpixels(p_replace = (0, 0.5), n_segments = (20, 100))), # Superpixels: 비슷한 색/밝기를 가진 pixel 그룹화
        iaa.Sometimes(0.5, iaa.LinearContrast((0.75, 1.5), per_channel = 0.5)), # LinearContrast: 이미지 일정 부분의 톤 차이를 돋보이게 만듦, 강한 대비
        iaa.Sometimes(0.5, iaa.Affine(shear = (-4, 4), # Affine: 이미지 회전
                                    rotate = (-40, 40), fit_output = True)), # fit_output: True 값을 적용해야 회전시켜도 이미지가 잘리지 않음
#         iaa.Sometimes(0.5, iaa.Emboss(alpha = (0, 0.5), strength = (0, 1.0))), # Emboss: 이미지 입체감 부여
#         iaa.OneOf([iaa.Grayscale(alpha = (0.0, 0.3)),
#                 iaa.MedianBlur(k = (3, 11))]), # MedianBlur: 이미지 블러 처리
        iaa.Sometimes(0.5, iaa.Sharpen(alpha = (0, 0.5), lightness = (0.75, 1.5)))], # Sharpen: 흐릿한 이미지를 선명하게 만듦
        random_order = True)

    image_aug, bbs_aug = seq(image = image, bounding_boxes = bbs) # image_aug : 증강된 이미지, bbs_aug : 증강이 적용된 bounding box 좌표

    cv2.imwrite(root_path + "aug_img1/"  + aug_name + img, image_aug) # 증강 이미지 저장
    # 증강된 사진 좌표
    resized = image_aug.shape
    rotated_size = [resized[1], resized[0]] # height, width -> width, height

    augmented_coords = after_box(bbs_aug)

    yolos = []

    for i in range(len(augmented_coords)):
        a = Pascal2Yolo(augmented_coords[i], rotated_size)
        final = cate + " " + " ".join(a)
        yolos.append(final)
  
    # print(yolos)
    # print("===================================================================================")


    f = open(root_path + "aug_label1/" + aug_name + label, "w", encoding = "utf-8", newline='') # 증강 라벨 저장
  
    for yolo in yolos:
        f.write(yolo)
        f.write("\n")
    f.close()
  

100%|██████████████████████████████████████████████████████████████████████████████| 1025/1025 [00:21<00:00, 47.52it/s]


### third

#### Categorize Augmented Data

In [10]:
aug_name = "aug_2_" # 원하는 파일명 적기

In [11]:
for img in tqdm(images):
#     print(img)
    image = cv2.imread(image_path + img) # 이미지 불러오기
#     print(image)
    label = img[:-4] + ".txt" # 라벨링 데이터 불러오기
#     print(label)
    height, width = image.shape[:2] # 원천 이미지 픽셀 불러오기 (height, weight, dimension)
    image_size = [width, height] # cv2는 height * weight를 출력하기 때문에 width, height형식으로 변환

    # bounding box 라벨 저장
    label_list = [] # 텍스트 파일에 있는 라벨을 담을 빈 리스트

    with open(label_path + label, "r") as f:
        labs = f.readlines()
        for lab in labs:
              label_list.append(lab.split()) # 라벨들을 리스트에 저장
        f.close()

    new_bounding_boxes = []

    for box in label_list:
        new_bounding_boxes.append(Yolo2Coord(image_size, box))
    cate = str(new_bounding_boxes[0][0])

    # augmentation 할 때 쓰는 BoundingBox 타입으로 변환
    ia_bounding_boxes = []

    for box in new_bounding_boxes:
        ia_bounding_boxes.append(ia.BoundingBox(x1 = box[1], y1 = box[2], x2 = box[3], y2 = box[4]))

    # BoundingBoxesOnImage([BoundingBox(x1= ~, y1= ~, x2= ~, y2= ~, label=None)형식

    # augmentation
    bbs = ia.BoundingBoxesOnImage(ia_bounding_boxes, shape = image.shape)

    seq = iaa.Sequential([
        iaa.AddToHueAndSaturation((-20, 20), per_channel = True),
        iaa.Sometimes(0.5, iaa.Invert(0.1, per_channel = True)),
        iaa.Sometimes(0.2, iaa.Grayscale(alpha = (0.0, 0.5))),
        iaa.OneOf([iaa.Flipud(0.3), # 30%의 이미지 상하 반전
                 iaa.Fliplr(0.4)]), # 40%의 이미지 좌우 반전
#         iaa.Sometimes(0.5, iaa.Superpixels(p_replace = (0, 0.5), n_segments = (20, 100))), # Superpixels: 비슷한 색/밝기를 가진 pixel 그룹화
        iaa.Sometimes(0.5, iaa.LinearContrast((0.75, 1.5), per_channel = 0.5)), # LinearContrast: 이미지 일정 부분의 톤 차이를 돋보이게 만듦, 강한 대비
        iaa.Sometimes(0.5, iaa.Affine(shear = (-4, 4), # Affine: 이미지 회전
                                    rotate = (-40, 40), fit_output = True)), # fit_output: True 값을 적용해야 회전시켜도 이미지가 잘리지 않음
#         iaa.Sometimes(0.5, iaa.Emboss(alpha = (0, 0.5), strength = (0, 1.0))), # Emboss: 이미지 입체감 부여
#         iaa.OneOf([iaa.Grayscale(alpha = (0.0, 0.3)),
#                 iaa.MedianBlur(k = (3, 11))]), # MedianBlur: 이미지 블러 처리
        iaa.Sometimes(0.5, iaa.Sharpen(alpha = (0, 0.5), lightness = (0.75, 1.5)))], # Sharpen: 흐릿한 이미지를 선명하게 만듦
        random_order = True)

    image_aug, bbs_aug = seq(image = image, bounding_boxes = bbs) # image_aug : 증강된 이미지, bbs_aug : 증강이 적용된 bounding box 좌표

    cv2.imwrite(root_path + "aug_img2/"  + aug_name + img, image_aug) # 증강 이미지 저장
    # 증강된 사진 좌표
    resized = image_aug.shape
    rotated_size = [resized[1], resized[0]] # height, width -> width, height

    augmented_coords = after_box(bbs_aug)

    yolos = []

    for i in range(len(augmented_coords)):
        a = Pascal2Yolo(augmented_coords[i], rotated_size)
        final = cate + " " + " ".join(a)
        yolos.append(final)
  
    # print(yolos)
    # print("===================================================================================")


    f = open(root_path + "aug_label2/" + aug_name + label, "w", encoding = "utf-8", newline='') # 증강 라벨 저장
  
    for yolo in yolos:
        f.write(yolo)
        f.write("\n")
    f.close()
  

100%|██████████████████████████████████████████████████████████████████████████████| 1025/1025 [00:21<00:00, 47.53it/s]


### fourth

#### Categorize Augmented Data

In [12]:
aug_name = "aug_3_" # 원하는 파일명 적기

In [13]:
for img in tqdm(images):
#     print(img)
    image = cv2.imread(image_path + img) # 이미지 불러오기
#     print(image)
    label = img[:-4] + ".txt" # 라벨링 데이터 불러오기
#     print(label)
    height, width = image.shape[:2] # 원천 이미지 픽셀 불러오기 (height, weight, dimension)
    image_size = [width, height] # cv2는 height * weight를 출력하기 때문에 width, height형식으로 변환

    # bounding box 라벨 저장
    label_list = [] # 텍스트 파일에 있는 라벨을 담을 빈 리스트

    with open(label_path + label, "r") as f:
        labs = f.readlines()
        for lab in labs:
              label_list.append(lab.split()) # 라벨들을 리스트에 저장
        f.close()

    new_bounding_boxes = []

    for box in label_list:
        new_bounding_boxes.append(Yolo2Coord(image_size, box))
    cate = str(new_bounding_boxes[0][0])

    # augmentation 할 때 쓰는 BoundingBox 타입으로 변환
    ia_bounding_boxes = []

    for box in new_bounding_boxes:
        ia_bounding_boxes.append(ia.BoundingBox(x1 = box[1], y1 = box[2], x2 = box[3], y2 = box[4]))

    # BoundingBoxesOnImage([BoundingBox(x1= ~, y1= ~, x2= ~, y2= ~, label=None)형식

    # augmentation
    bbs = ia.BoundingBoxesOnImage(ia_bounding_boxes, shape = image.shape)

    seq = iaa.Sequential([
        iaa.AddToHueAndSaturation((-20, 20), per_channel = True),
        iaa.Sometimes(0.5, iaa.Invert(0.1, per_channel = True)),
        iaa.Sometimes(0.2, iaa.Grayscale(alpha = (0.0, 0.5))),
        iaa.OneOf([iaa.Flipud(0.3), # 30%의 이미지 상하 반전
                 iaa.Fliplr(0.4)]), # 40%의 이미지 좌우 반전
#         iaa.Sometimes(0.5, iaa.Superpixels(p_replace = (0, 0.5), n_segments = (20, 100))), # Superpixels: 비슷한 색/밝기를 가진 pixel 그룹화
        iaa.Sometimes(0.5, iaa.LinearContrast((0.75, 1.5), per_channel = 0.5)), # LinearContrast: 이미지 일정 부분의 톤 차이를 돋보이게 만듦, 강한 대비
        iaa.Sometimes(0.5, iaa.Affine(shear = (-4, 4), # Affine: 이미지 회전
                                    rotate = (-40, 40), fit_output = True)), # fit_output: True 값을 적용해야 회전시켜도 이미지가 잘리지 않음
#         iaa.Sometimes(0.5, iaa.Emboss(alpha = (0, 0.5), strength = (0, 1.0))), # Emboss: 이미지 입체감 부여
#         iaa.OneOf([iaa.Grayscale(alpha = (0.0, 0.3)),
#                 iaa.MedianBlur(k = (3, 11))]), # MedianBlur: 이미지 블러 처리
        iaa.Sometimes(0.5, iaa.Sharpen(alpha = (0, 0.5), lightness = (0.75, 1.5)))], # Sharpen: 흐릿한 이미지를 선명하게 만듦
        random_order = True)

    image_aug, bbs_aug = seq(image = image, bounding_boxes = bbs) # image_aug : 증강된 이미지, bbs_aug : 증강이 적용된 bounding box 좌표

    cv2.imwrite(root_path + "aug_img3/"  + aug_name + img, image_aug) # 증강 이미지 저장
    # 증강된 사진 좌표
    resized = image_aug.shape
    rotated_size = [resized[1], resized[0]] # height, width -> width, height

    augmented_coords = after_box(bbs_aug)

    yolos = []

    for i in range(len(augmented_coords)):
        a = Pascal2Yolo(augmented_coords[i], rotated_size)
        final = cate + " " + " ".join(a)
        yolos.append(final)
  
    # print(yolos)
    # print("===================================================================================")


    f = open(root_path + "aug_label3/" + aug_name + label, "w", encoding = "utf-8", newline='') # 증강 라벨 저장
  
    for yolo in yolos:
        f.write(yolo)
        f.write("\n")
    f.close()
  

100%|██████████████████████████████████████████████████████████████████████████████| 1025/1025 [00:20<00:00, 49.72it/s]


# SH LIM

## Route Setting

In [14]:
# 경로 설정
root_path = "C:/HUFS_capstone/dataset/SHLIM/Dataset_SH/" #원천 이미지 & 라벨링 디렉토리
image_path = root_path + "train_images(origin)/" # 원천 이미지 디렉토리
label_path = root_path + "train_labels(origin)/" # 원천 이미지 라벨 디렉토리

images = list(os.listdir(image_path))
print(len(images), images[:5], sep="\n")
labels = list(os.listdir(label_path))
print(len(labels), labels[:5], sep="\n")

1004
['KakaoTalk_20221103_225834284.jpg', 'KakaoTalk_20221103_225834284_01.jpg', 'KakaoTalk_20221103_225834284_02.jpg', 'KakaoTalk_20221103_225834284_03.jpg', 'KakaoTalk_20221103_225834284_04.jpg']
1004
['KakaoTalk_20221103_225834284.txt', 'KakaoTalk_20221103_225834284_01.txt', 'KakaoTalk_20221103_225834284_02.txt', 'KakaoTalk_20221103_225834284_03.txt', 'KakaoTalk_20221103_225834284_04.txt']


## Augmentation

## first

#### Categorize Augmented Data

In [15]:
aug_name = "aug__" # 원하는 파일명 적기

In [16]:
for img in tqdm(images):
#     print(img)
    image = cv2.imread(image_path + img) # 이미지 불러오기
#     print(image_path + img)
    label = img[:-4] + ".txt" # 라벨링 데이터 불러오기
#     print(label)
    height, width = image.shape[:2] # 원천 이미지 픽셀 불러오기 (height, weight, dimension)
    image_size = [width, height] # cv2는 height * weight를 출력하기 때문에 width, height형식으로 변환

    # bounding box 라벨 저장
    label_list = [] # 텍스트 파일에 있는 라벨을 담을 빈 리스트

    with open(label_path + label, "r") as f:
        labs = f.readlines()
        for lab in labs:
              label_list.append(lab.split()) # 라벨들을 리스트에 저장
        f.close()

    new_bounding_boxes = []

    for box in label_list:
        new_bounding_boxes.append(Yolo2Coord(image_size, box))
    cate = str(new_bounding_boxes[0][0])

    # augmentation 할 때 쓰는 BoundingBox 타입으로 변환
    ia_bounding_boxes = []

    for box in new_bounding_boxes:
        ia_bounding_boxes.append(ia.BoundingBox(x1 = box[1], y1 = box[2], x2 = box[3], y2 = box[4]))

    # BoundingBoxesOnImage([BoundingBox(x1= ~, y1= ~, x2= ~, y2= ~, label=None)형식

    # augmentation
    bbs = ia.BoundingBoxesOnImage(ia_bounding_boxes, shape = image.shape)

    seq = iaa.Sequential([
        iaa.AddToHueAndSaturation((-20, 20), per_channel = True),
        iaa.Sometimes(0.5, iaa.Invert(0.1, per_channel = True)),
        iaa.Sometimes(0.2, iaa.Grayscale(alpha = (0.0, 0.5))),
        iaa.OneOf([iaa.Flipud(0.3), # 30%의 이미지 상하 반전
                 iaa.Fliplr(0.4)]), # 40%의 이미지 좌우 반전
#         iaa.Sometimes(0.5, iaa.Superpixels(p_replace = (0, 0.5), n_segments = (20, 100))), # Superpixels: 비슷한 색/밝기를 가진 pixel 그룹화
        iaa.Sometimes(0.5, iaa.LinearContrast((0.75, 1.5), per_channel = 0.5)), # LinearContrast: 이미지 일정 부분의 톤 차이를 돋보이게 만듦, 강한 대비
        iaa.Sometimes(0.5, iaa.Affine(shear = (-4, 4), # Affine: 이미지 회전
                                    rotate = (-40, 40), fit_output = True)), # fit_output: True 값을 적용해야 회전시켜도 이미지가 잘리지 않음
#         iaa.Sometimes(0.5, iaa.Emboss(alpha = (0, 0.5), strength = (0, 1.0))), # Emboss: 이미지 입체감 부여
#         iaa.OneOf([iaa.Grayscale(alpha = (0.0, 0.3)),
#                 iaa.MedianBlur(k = (3, 11))]), # MedianBlur: 이미지 블러 처리
        iaa.Sometimes(0.5, iaa.Sharpen(alpha = (0, 0.5), lightness = (0.75, 1.5)))], # Sharpen: 흐릿한 이미지를 선명하게 만듦
        random_order = True)

    image_aug, bbs_aug = seq(image = image, bounding_boxes = bbs) # image_aug : 증강된 이미지, bbs_aug : 증강이 적용된 bounding box 좌표

    cv2.imwrite(root_path + "aug_img/"  + aug_name + img, image_aug) # 증강 이미지 저장
    # 증강된 사진 좌표
    resized = image_aug.shape
    rotated_size = [resized[1], resized[0]] # height, width -> width, height

    augmented_coords = after_box(bbs_aug)

    yolos = []

    for i in range(len(augmented_coords)):
        a = Pascal2Yolo(augmented_coords[i], rotated_size)
        final = cate + " " + " ".join(a)
        yolos.append(final)
  
    # print(yolos)
    # print("===================================================================================")


    f = open(root_path + "aug_label/" + aug_name + label, "w", encoding = "utf-8", newline='') # 증강 라벨 저장
  
    for yolo in yolos:
        f.write(yolo)
        f.write("\n")
    f.close()
  

100%|██████████████████████████████████████████████████████████████████████████████| 1004/1004 [05:08<00:00,  3.25it/s]


## second

#### Categorize Augmented Data

In [17]:
aug_name = "aug_1_" # 원하는 파일명 적기

In [18]:
for img in tqdm(images):
#     print(img)
    image = cv2.imread(image_path + img) # 이미지 불러오기
#     print(image)
    label = img[:-4] + ".txt" # 라벨링 데이터 불러오기
#     print(label)
    height, width = image.shape[:2] # 원천 이미지 픽셀 불러오기 (height, weight, dimension)
    image_size = [width, height] # cv2는 height * weight를 출력하기 때문에 width, height형식으로 변환

    # bounding box 라벨 저장
    label_list = [] # 텍스트 파일에 있는 라벨을 담을 빈 리스트

    with open(label_path + label, "r") as f:
        labs = f.readlines()
        for lab in labs:
              label_list.append(lab.split()) # 라벨들을 리스트에 저장
        f.close()

    new_bounding_boxes = []

    for box in label_list:
        new_bounding_boxes.append(Yolo2Coord(image_size, box))
    cate = str(new_bounding_boxes[0][0])

    # augmentation 할 때 쓰는 BoundingBox 타입으로 변환
    ia_bounding_boxes = []

    for box in new_bounding_boxes:
        ia_bounding_boxes.append(ia.BoundingBox(x1 = box[1], y1 = box[2], x2 = box[3], y2 = box[4]))

    # BoundingBoxesOnImage([BoundingBox(x1= ~, y1= ~, x2= ~, y2= ~, label=None)형식

    # augmentation
    bbs = ia.BoundingBoxesOnImage(ia_bounding_boxes, shape = image.shape)

    seq = iaa.Sequential([
        iaa.AddToHueAndSaturation((-20, 20), per_channel = True),
        iaa.Sometimes(0.5, iaa.Invert(0.1, per_channel = True)),
        iaa.Sometimes(0.2, iaa.Grayscale(alpha = (0.0, 0.5))),
        iaa.OneOf([iaa.Flipud(0.3), # 30%의 이미지 상하 반전
                 iaa.Fliplr(0.4)]), # 40%의 이미지 좌우 반전
#         iaa.Sometimes(0.5, iaa.Superpixels(p_replace = (0, 0.5), n_segments = (20, 100))), # Superpixels: 비슷한 색/밝기를 가진 pixel 그룹화
        iaa.Sometimes(0.5, iaa.LinearContrast((0.75, 1.5), per_channel = 0.5)), # LinearContrast: 이미지 일정 부분의 톤 차이를 돋보이게 만듦, 강한 대비
        iaa.Sometimes(0.5, iaa.Affine(shear = (-4, 4), # Affine: 이미지 회전
                                    rotate = (-40, 40), fit_output = True)), # fit_output: True 값을 적용해야 회전시켜도 이미지가 잘리지 않음
#         iaa.Sometimes(0.5, iaa.Emboss(alpha = (0, 0.5), strength = (0, 1.0))), # Emboss: 이미지 입체감 부여
#         iaa.OneOf([iaa.Grayscale(alpha = (0.0, 0.3)),
#                 iaa.MedianBlur(k = (3, 11))]), # MedianBlur: 이미지 블러 처리
        iaa.Sometimes(0.5, iaa.Sharpen(alpha = (0, 0.5), lightness = (0.75, 1.5)))], # Sharpen: 흐릿한 이미지를 선명하게 만듦
        random_order = True)

    image_aug, bbs_aug = seq(image = image, bounding_boxes = bbs) # image_aug : 증강된 이미지, bbs_aug : 증강이 적용된 bounding box 좌표

    cv2.imwrite(root_path + "aug_img1/"  + aug_name + img, image_aug) # 증강 이미지 저장
    # 증강된 사진 좌표
    resized = image_aug.shape
    rotated_size = [resized[1], resized[0]] # height, width -> width, height

    augmented_coords = after_box(bbs_aug)

    yolos = []

    for i in range(len(augmented_coords)):
        a = Pascal2Yolo(augmented_coords[i], rotated_size)
        final = cate + " " + " ".join(a)
        yolos.append(final)
  
    # print(yolos)
    # print("===================================================================================")


    f = open(root_path + "aug_label1/" + aug_name + label, "w", encoding = "utf-8", newline='') # 증강 라벨 저장
  
    for yolo in yolos:
        f.write(yolo)
        f.write("\n")
    f.close()
  

100%|██████████████████████████████████████████████████████████████████████████████| 1004/1004 [04:58<00:00,  3.37it/s]


### third

#### Categorize Augmented Data

In [19]:
aug_name = "aug_2_" # 원하는 파일명 적기

In [20]:
for img in tqdm(images):
#     print(img)
    image = cv2.imread(image_path + img) # 이미지 불러오기
#     print(image)
    label = img[:-4] + ".txt" # 라벨링 데이터 불러오기
#     print(label)
    height, width = image.shape[:2] # 원천 이미지 픽셀 불러오기 (height, weight, dimension)
    image_size = [width, height] # cv2는 height * weight를 출력하기 때문에 width, height형식으로 변환

    # bounding box 라벨 저장
    label_list = [] # 텍스트 파일에 있는 라벨을 담을 빈 리스트

    with open(label_path + label, "r") as f:
        labs = f.readlines()
        for lab in labs:
              label_list.append(lab.split()) # 라벨들을 리스트에 저장
        f.close()

    new_bounding_boxes = []

    for box in label_list:
        new_bounding_boxes.append(Yolo2Coord(image_size, box))
    cate = str(new_bounding_boxes[0][0])

    # augmentation 할 때 쓰는 BoundingBox 타입으로 변환
    ia_bounding_boxes = []

    for box in new_bounding_boxes:
        ia_bounding_boxes.append(ia.BoundingBox(x1 = box[1], y1 = box[2], x2 = box[3], y2 = box[4]))

    # BoundingBoxesOnImage([BoundingBox(x1= ~, y1= ~, x2= ~, y2= ~, label=None)형식

    # augmentation
    bbs = ia.BoundingBoxesOnImage(ia_bounding_boxes, shape = image.shape)

    seq = iaa.Sequential([
        iaa.AddToHueAndSaturation((-20, 20), per_channel = True),
        iaa.Sometimes(0.5, iaa.Invert(0.1, per_channel = True)),
        iaa.Sometimes(0.2, iaa.Grayscale(alpha = (0.0, 0.5))),
        iaa.OneOf([iaa.Flipud(0.3), # 30%의 이미지 상하 반전
                 iaa.Fliplr(0.4)]), # 40%의 이미지 좌우 반전
#         iaa.Sometimes(0.5, iaa.Superpixels(p_replace = (0, 0.5), n_segments = (20, 100))), # Superpixels: 비슷한 색/밝기를 가진 pixel 그룹화
        iaa.Sometimes(0.5, iaa.LinearContrast((0.75, 1.5), per_channel = 0.5)), # LinearContrast: 이미지 일정 부분의 톤 차이를 돋보이게 만듦, 강한 대비
        iaa.Sometimes(0.5, iaa.Affine(shear = (-4, 4), # Affine: 이미지 회전
                                    rotate = (-40, 40), fit_output = True)), # fit_output: True 값을 적용해야 회전시켜도 이미지가 잘리지 않음
#         iaa.Sometimes(0.5, iaa.Emboss(alpha = (0, 0.5), strength = (0, 1.0))), # Emboss: 이미지 입체감 부여
#         iaa.OneOf([iaa.Grayscale(alpha = (0.0, 0.3)),
#                 iaa.MedianBlur(k = (3, 11))]), # MedianBlur: 이미지 블러 처리
        iaa.Sometimes(0.5, iaa.Sharpen(alpha = (0, 0.5), lightness = (0.75, 1.5)))], # Sharpen: 흐릿한 이미지를 선명하게 만듦
        random_order = True)

    image_aug, bbs_aug = seq(image = image, bounding_boxes = bbs) # image_aug : 증강된 이미지, bbs_aug : 증강이 적용된 bounding box 좌표

    cv2.imwrite(root_path + "aug_img2/"  + aug_name + img, image_aug) # 증강 이미지 저장
    # 증강된 사진 좌표
    resized = image_aug.shape
    rotated_size = [resized[1], resized[0]] # height, width -> width, height

    augmented_coords = after_box(bbs_aug)

    yolos = []

    for i in range(len(augmented_coords)):
        a = Pascal2Yolo(augmented_coords[i], rotated_size)
        final = cate + " " + " ".join(a)
        yolos.append(final)
  
    # print(yolos)
    # print("===================================================================================")


    f = open(root_path + "aug_label2/" + aug_name + label, "w", encoding = "utf-8", newline='') # 증강 라벨 저장
  
    for yolo in yolos:
        f.write(yolo)
        f.write("\n")
    f.close()
  

100%|██████████████████████████████████████████████████████████████████████████████| 1004/1004 [04:48<00:00,  3.49it/s]


### fourth

#### Categorize Augmented Data

In [21]:
aug_name = "aug_3_" # 원하는 파일명 적기

In [22]:
for img in tqdm(images):
#     print(img)
    image = cv2.imread(image_path + img) # 이미지 불러오기
#     print(image)
    label = img[:-4] + ".txt" # 라벨링 데이터 불러오기
#     print(label)
    height, width = image.shape[:2] # 원천 이미지 픽셀 불러오기 (height, weight, dimension)
    image_size = [width, height] # cv2는 height * weight를 출력하기 때문에 width, height형식으로 변환

    # bounding box 라벨 저장
    label_list = [] # 텍스트 파일에 있는 라벨을 담을 빈 리스트

    with open(label_path + label, "r") as f:
        labs = f.readlines()
        for lab in labs:
              label_list.append(lab.split()) # 라벨들을 리스트에 저장
        f.close()

    new_bounding_boxes = []

    for box in label_list:
        new_bounding_boxes.append(Yolo2Coord(image_size, box))
    cate = str(new_bounding_boxes[0][0])

    # augmentation 할 때 쓰는 BoundingBox 타입으로 변환
    ia_bounding_boxes = []

    for box in new_bounding_boxes:
        ia_bounding_boxes.append(ia.BoundingBox(x1 = box[1], y1 = box[2], x2 = box[3], y2 = box[4]))

    # BoundingBoxesOnImage([BoundingBox(x1= ~, y1= ~, x2= ~, y2= ~, label=None)형식

    # augmentation
    bbs = ia.BoundingBoxesOnImage(ia_bounding_boxes, shape = image.shape)

    seq = iaa.Sequential([
        iaa.AddToHueAndSaturation((-20, 20), per_channel = True),
        iaa.Sometimes(0.5, iaa.Invert(0.1, per_channel = True)),
        iaa.Sometimes(0.2, iaa.Grayscale(alpha = (0.0, 0.5))),
        iaa.OneOf([iaa.Flipud(0.3), # 30%의 이미지 상하 반전
                 iaa.Fliplr(0.4)]), # 40%의 이미지 좌우 반전
#         iaa.Sometimes(0.5, iaa.Superpixels(p_replace = (0, 0.5), n_segments = (20, 100))), # Superpixels: 비슷한 색/밝기를 가진 pixel 그룹화
        iaa.Sometimes(0.5, iaa.LinearContrast((0.75, 1.5), per_channel = 0.5)), # LinearContrast: 이미지 일정 부분의 톤 차이를 돋보이게 만듦, 강한 대비
        iaa.Sometimes(0.5, iaa.Affine(shear = (-4, 4), # Affine: 이미지 회전
                                    rotate = (-40, 40), fit_output = True)), # fit_output: True 값을 적용해야 회전시켜도 이미지가 잘리지 않음
#         iaa.Sometimes(0.5, iaa.Emboss(alpha = (0, 0.5), strength = (0, 1.0))), # Emboss: 이미지 입체감 부여
#         iaa.OneOf([iaa.Grayscale(alpha = (0.0, 0.3)),
#                 iaa.MedianBlur(k = (3, 11))]), # MedianBlur: 이미지 블러 처리
        iaa.Sometimes(0.5, iaa.Sharpen(alpha = (0, 0.5), lightness = (0.75, 1.5)))], # Sharpen: 흐릿한 이미지를 선명하게 만듦
        random_order = True)

    image_aug, bbs_aug = seq(image = image, bounding_boxes = bbs) # image_aug : 증강된 이미지, bbs_aug : 증강이 적용된 bounding box 좌표

    cv2.imwrite(root_path + "aug_img3/"  + aug_name + img, image_aug) # 증강 이미지 저장
    # 증강된 사진 좌표
    resized = image_aug.shape
    rotated_size = [resized[1], resized[0]] # height, width -> width, height

    augmented_coords = after_box(bbs_aug)

    yolos = []

    for i in range(len(augmented_coords)):
        a = Pascal2Yolo(augmented_coords[i], rotated_size)
        final = cate + " " + " ".join(a)
        yolos.append(final)
  
    # print(yolos)
    # print("===================================================================================")


    f = open(root_path + "aug_label3/" + aug_name + label, "w", encoding = "utf-8", newline='') # 증강 라벨 저장
  
    for yolo in yolos:
        f.write(yolo)
        f.write("\n")
    f.close()
  

100%|██████████████████████████████████████████████████████████████████████████████| 1004/1004 [04:48<00:00,  3.48it/s]
